介绍参数管理(parameters)  

    访问参数，用于调试、诊断和可视化；

    参数初始化；

    在不同模型组件间共享参数。


训练阶段我们的目标是找到使损失函数最小化的模型参数值  
经过训练后，我们将需要使用这些参数来做出未来的预测。  
此外，有时我们希望提取参数，以便在其他环境中复用它们   
将模型保存下来，以便它可以在其他软件中执行， 或者为了获得科学的理解而进行检查。

In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)


tensor([[0.0901],
        [0.1023]], grad_fn=<AddmmBackward0>)

从已有模型中访问参数  
当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。  
这就像模型是一个列表一样，每层的参数都在其属性中。 

In [3]:
#检查第二个全连接层的参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0521,  0.2268, -0.3199,  0.3117, -0.3089,  0.2409, -0.0738, -0.0154]])), ('bias', tensor([0.1458]))])


全连接层包含两个参数，分别是该层的权重和偏置。  
两者都存储为单精度浮点数（float32）。  
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

每个参数都表示为参数类的一个实例。  
要对参数执行任何操作，首先我们需要访问底层的数值。  
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。  
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置， 提取后返回的是一个参数类实例，并进一步访问该参数的值。

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)


<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1458], requires_grad=True)
tensor([0.1458])


参数是复合的对象，包含值、梯度和额外信息。   
除了值之外，我们还可以访问每个参数的梯度。  
在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

In [5]:
net[2].weight.grad==None

True

一次性访问所有参数  
当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。  
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂， 因为我们需要递归整个树来提取每个子块的参数。

In [6]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式，如下所示

In [7]:
net.state_dict()['2.bias'].data

tensor([0.1458])

从嵌套块收集参数  
如果我们将多个块相互嵌套，参数命名约定是如何工作的。  
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)


tensor([[0.2015],
        [0.2011]], grad_fn=<AddmmBackward0>)

In [10]:
#设计了网络后，我们看看它是如何工作的
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。  
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [11]:
rgnet[0][1][0].bias.data

tensor([-0.0400,  0.3567, -0.0385,  0.2286,  0.1470, -0.2588,  0.4917, -0.3012])

参数初始化  
看看如何正确地初始化参数  
深度学习框架提供默认随机初始化， 也允许我们创建自定义初始化方法， 满足我们通过其他规则实现初始化权重。  
默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵， 这个范围是根据输入和输出维度计算出的。   
PyTorch的nn.init模块提供了多种预置初始化方法。

内置的初始化器  
 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量， 且将偏置参数设置为0。

In [12]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([-0.0043,  0.0100,  0.0076, -0.0164]), tensor(0.))

还可以将所有参数初始化为给定的常数，比如初始化为1

In [13]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

还可以对某些块应用不同的初始化方法。  
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42。

In [14]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


tensor([ 0.6298, -0.3628,  0.3677, -0.0098])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化  
有时，深度学习框架没有提供我们需要的初始化方法。  
在下面的例子中，我们使用以下的分布为任意权重参数w定义初始化方法：
$$
w \sim \begin{cases} 
U(5, 10) & \text{可能性 } \frac{1}{4} \\ 
0 & \text{可能性 } \frac{1}{2} \\ 
U(-10, -5) & \text{可能性 } \frac{1}{4} 
\end{cases}
$$
同样，我们实现了一个my_init函数来应用到net

In [15]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000,  8.8908, -6.2080],
        [ 7.2066, -8.4546,  9.4437,  8.5047]], grad_fn=<SliceBackward0>)

始终可以直接设置参数

In [16]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]


tensor([42.0000,  1.0000,  9.8908, -5.2080])

参数绑定  
希望在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数

In [17]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第三个和第五个神经网络层的参数是绑定的。  
它们不仅值相等，而且由相同的张量表示。  
因此，如果我们改变其中一个参数，另一个参数也会改变。  
这里有一个问题：当参数绑定时，梯度会发生什么情况？  
答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

延后初始化

<pre>到目前为止，我们忽略了建立网络时需要做的以下这些事情：

    我们定义了网络架构，但没有指定输入维度。

    我们添加层时没有指定前一层的输出维度。

    我们在初始化参数时，甚至没有足够的信息来确定模型应该包含多少参数。

有些读者可能会对我们的代码能运行感到惊讶。  
毕竟，深度学习框架无法判断网络的输入维度是什么。  
这里的诀窍是框架的延后初始化（defers initialization）， 即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

在以后，当使用卷积神经网络时， 由于输入维度（即图像的分辨率）将影响每个后续层的维数， 有了该技术将更加方便。  
现在我们在编写代码时无须知道维度是什么就可以设置参数， 这种能力可以大大简化定义和修改模型的任务。  
接下来，我们将更深入地研究初始化机制。

In [18]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(),nn.Linear(256,10))
print(net)

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


此时，因为输入维数是未知的，所以网络不可能知道输入层权重的维数。 因此，框架尚未初始化任何参数，我们通过尝试访问以下参数进行确认。

In [19]:
[net[i].state_dict() for i in range(len(net))]

[OrderedDict([('weight', <UninitializedParameter>),
              ('bias', <UninitializedParameter>)]),
 OrderedDict(),
 OrderedDict([('weight',
               tensor([[ 0.0449,  0.0394, -0.0511,  ...,  0.0039,  0.0350, -0.0249],
                       [ 0.0485,  0.0060,  0.0228,  ..., -0.0566,  0.0102, -0.0506],
                       [ 0.0361, -0.0016,  0.0534,  ..., -0.0305, -0.0582,  0.0435],
                       ...,
                       [ 0.0342,  0.0600,  0.0021,  ...,  0.0354, -0.0064, -0.0341],
                       [-0.0012, -0.0601,  0.0427,  ..., -0.0443, -0.0520,  0.0045],
                       [ 0.0008,  0.0589,  0.0388,  ..., -0.0622,  0.0411,  0.0374]])),
              ('bias',
               tensor([ 0.0237, -0.0101, -0.0477, -0.0237, -0.0096,  0.0117,  0.0310,  0.0513,
                       -0.0368,  0.0448]))])]

接下来让我们将数据通过网络，最终使框架初始化参数。

In [20]:
low = torch.finfo(torch.float32).min/10
high = torch.finfo(torch.float32).max/10
X = torch.zeros([2,20],dtype=torch.float32).uniform_(low, high)

net(X)
print(net)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
